In [ ]:
# do all necessary imports for this chapter
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from scipy.sparse.linalg import eigs, LinearOperator
from ncon import ncon
from tutorialFunctions import createMPS, mixedCanonical, minAcC
from time import time

# [Tangent-space methods for uniform matrix product states](https://doi.org/10.21468/SciPostPhysLectNotes.7)

## 3. Transfer matrices and fixed points

### 3.1 MPS as fixed points of one-dimensional transfer matrices

Matrix product states have been used extensively as a variational ansatz for ground states of local Hamiltonians. In recent years, it has been observed that they can also provide accurate approximations for fixed points of transfer matrices. In this notebook we investigate how tangent-space methods for MPS can be applied to one-dimensional transfer matrices.

A one-dimensional transfer matrix in the form of a *matrix product operator* (MPO) is given by

$$
T(O) = \sum_{\{i\}, \{j\}} \left( \dots O^{i_{n-1}, j_{n-1}} O^{i_{n}, j_{n}} O^{i_{n+1}, j_{n+1}} \dots \right)
\left | i_{n-1} \right \rangle \left \langle j_{n-1} \right | \otimes \left | i_{n} \right \rangle \left \langle j_{n} \right | \otimes \left | i_{n+1} \right \rangle \left \langle j_{n+1} \right | \dots \;,
$$

which can be represented diagrammatically as

<center><img src="./img/transferMpo.svg" alt="transferMpo"/></center>

Such an object naturally arises in the context of infinite two-dimensional tensor networks, which can be interpreted as an infinite power of a corresponding one-dimensional row-to-row transfer matrix. This means that the contraction of the network is equivalent to finding the leading eigenvector $\left | \Psi \right \rangle$, referred to as the *fixed point*, of the transfer matrix which satisfies the equation

$$T(O) \left | \Psi \right \rangle \propto \left | \Psi \right \rangle.$$

We can now propose an MPS ansatz for this fixed point, such that it obeys the eigenvalue equation

<center><img src="./img/fixedPoint.svg" alt="fixedPoint"/></center>

This MPS fixed point can be computed through a variation on the VUMPS algorithm introduced in the previous chapter, as will be explained in the next section. Suppose for now that we have managed to find an MPS representation $\left | \Psi(A) \right \rangle$ of the fixed point of $T(O)$. The corresponding eigenvalue $\Lambda$ is then given by

$$ \Lambda = \left \langle \Psi(\bar{A}) \middle | T \middle | \Psi(A) \right \rangle , $$

assuming as always that we are dealing with a properly normalized MPS. If we bring $\left | \Psi(A) \right \rangle$ in mixed canonical form, then $\Lambda$ is given by the network

<center><img src="./img/lambda.svg" alt="lambda"/></center>

We can contract this resulting infinite network by finding the fixed points of the left and right channel operators $T_L$ and $T_R$ which are defined as

<center><img src="./img/channels.svg" alt="channels"/></center>

The corresponding fixed points $F_L$ and $F_R$, also referred to as the left and right *environments*, satisfy

<center><img src="./img/environments.svg" alt="environments"/></center>

and can be normalized such that

<center><img src="./img/envNorm.svg" alt="envNorm"/></center>

The eigenvalues $\lambda_L$ and $\lambda_R$ have to correspond to the same value $\lambda$ by construction, so that $\Lambda$ is given by

$$ \Lambda = \lim_{N \to \infty} \lambda^N ,$$

where $N$ is the number of sites in the horizontal direction. Finally, we note that we can associate a *free energy density* $f = -\frac{1}{N} \log \Lambda = -\log \lambda$ to this MPS fixed point.


### 3.2 The VUMPS algorithm for MPOs

In order to formulate an algorithm for finding this MPS fixed point we start by stating the optimality condition it must satisfy in order to qualify as an approximate eigenvector of $T(O)$. Intuitively, what we would like to impose is that the residual $T(O) \left| \Psi \right \rangle - \Lambda \left | \Psi \right \rangle$ is equal to zero. While this condition can never be satisfied exactly for any MPS approximation, we can however demand that the tangent space projection of this residual vanishes,

$$
\mathcal{P}_A \left( T(O) \left| \Psi \right \rangle - \Lambda \left | \Psi \right \rangle \right) = 0,
$$

where $\mathcal{P}_A$ is the projector onto the tangent space to the MPS manifold at $A$. Similar to the Hamiltonian case, this projected residual can be characterized in terms of a tangent space gradient $G$,

$$
G = A_C' - A_L C' = A_c' - C' A_R,
$$

where $A_C'$ and $C'$ are now given by

<center><img src="./img/AcPrime2.svg" alt="AcPrime2"/></center>

and

<center><img src="./img/CPrime2.svg" alt="CPrime2"/></center>

The optimality condition for the fixed point MPS is then equivalent to having $||G|| = 0$. In addition, if the MPO defining the transfer matrix is hermitian then it can be shown that the optimality condition corresponds to the variational minimum of the free energy density introduced above. Similar to the Hamiltonian case, if we introduce operators $O_{A_C}(\cdot)$ and $O_C(\cdot)$ such that

$$
\begin{align}
O_{A_C}(A_C) = A_C', \\
O_{C}(C) = C',
\end{align}
$$

then it follows that the fixed point is characterized by the set of equations

$$
\begin{align}
O_{A_C}(A_C) \propto A_C, \\
O_{C}(C) \propto C, \\
A_C = A_L C = C A_R.
\end{align}
$$

The VUMPS algorithm for MPOs then corresponds to an iterative scheme for finding the solutions to these equations starting from a given set $\{A_L, A_C, A_R, C\}$ which consists of the following steps:

1. Find the left and right environments $F_L$ and $F_R$ and use these to solve the eigenvalue equations for $O_{A_C}$ and $O_C$, giving new center tensors $\tilde{A}_C$ and $\tilde{C}$.

2. From these new center tensors, extract the $\tilde{A}_L$ and $\tilde{A}_R$ that minimize $||\tilde{A}_C - \tilde{A}_L \tilde{C}||$ and $||\tilde{A}_C - \tilde{C} \tilde{A}_L||$ using the `minAcC` routine from the previous chapter.

3. Update the set of tensors $\{A_L, A_C, A_R, C\} \leftarrow \{\tilde{A}_L, \tilde{A}_C, \tilde{A}_R, \tilde{C}\}$ and evaluate the optimality condition $\varepsilon = \left | \left | O_{A_C} (A_C) - A_L O_C(C) \right | \right |$.

4. If the optimality condition lies above the given tolerance, repeat.

#### Implementing the VUMPS algorithm

We start by implementing the routines for finding and normalizing the left and right environments of the channel operators.

In [ ]:
def leftEnvironment(O, Al, tol):
    """
    Computes the left environment as the fixed point of the left channel operator.

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        tol : float, optional
            tolerance for eigenvalue solver

        Returns
        -------
        lam : float
            Leading left eigenvalue.
        Fl : np.array (D, d, D)
            left environment,
            ordered bottom-middle-top.

    """
    
    # given as an example
    
    D = Al.shape[0]
    d = Al.shape[1]
    tol = max(tol, 1e-14)
    
    # construct handle for the action of the relevant operator and cast to linear operator
    channelLeftHandle = lambda v: (ncon((v.reshape((D,d,D)), Al, np.conj(Al), O),([5, 3, 1], [1, 2, -3], [5, 4, -1], [3, 2, -2, 4]))).reshape(-1)
    channelLeft = LinearOperator((D**2*d, D**2*d), matvec=channelLeftHandle)
    # compute the largest magnitude eigenvalue and corresponding eigenvector
    lam, Fl = eigs(channelLeft, k=1, which="LM", tol=tol)
    
    return lam[0], Fl.reshape((D,d,D))


def rightEnvironment(O, Ar, tol):
    """
    Computes the right environment as the fixed point of the right channel operator.

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        tol : float, optional
            tolerance for eigenvalue solver

        Returns
        -------
        lam : float
            Leading right eigenvalue.
        Fr : np.array (D, d, D)
            right environment,
            ordered top-middle-bottom.

    """
    
    return lam, Fr


def environments(O, Al, Ar, C, tol=1e-5):
    """
    Compute the left and right environments of the channel operators
    as well as the corresponding eigenvalue.

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        tol : float, optional
            tolerance for eigenvalue solver

        Returns
        -------
        lam : float
            Leading eigenvalue of the channel
            operators.
        Fl : np.array (D, d, D)
            left environment,
            ordered bottom-middle-top.
        Fr : np.array (D, d, D)
            right environment,
            ordered top-middle-bottom.

    """
    
    return lam, Fl, Fr

Next we implement the action of the effective operators $O_{A_C}$ and $O_C$ on a given input tensor,

<center><img src="./img/O_Ac.svg" alt="O_Ac"/></center>

and

<center><img src="./img/O_C.svg" alt="O_C"/></center>

In [ ]:
def O_Ac(x, O, Fl, Fr, lam):
    """
    Action of the operator O_Ac on a given tensor.

        Parameters
        ----------
        x : np.array (D, d, D)
            Tensor of size (D, d, D)
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Fl : np.array (D, d, D)
            left environment,
            ordered bottom-middle-top.
        Fr : np.array (D, d, D)
            right environment,
            ordered top-middle-bottom.
        lam : float
            Leading eigenvalue.

        Returns
        -------
        y : np.array (D, d, D)
            Result of the action of O_Ac on the tensor x.

    """
    
    # given as an example
    
    y = ncon((Fl, Fr, x, O),([-1, 2, 1], [4, 5, -3], [1, 3, 4], [2, 3, 5, -2])) / lam
    
    return y


def O_C(x, Fl, Fr):
    """
    Action of the operator O_C on a given tensor.

        Parameters
        ----------
        x : np.array (D, D)
            Tensor of size (D, D)
        Fl : np.array (D, d, D)
            left environment,
            ordered bottom-middle-top.
        Fr : np.array (D, d, D)
            right environment,
            ordered top-middle-bottom.

        Returns
        -------
        y : np.array (D, d, D)
            Result of the action of O_C on the tensor x.

    """
    
    return y

This then allows to define a new routine `calcNewCenterMpo` which finds the new center tensors $\tilde{A}_C$ and $\tilde{C}$ by solving the eigenvalue problems for $O_{A_C}$ and $O_C$.

In [ ]:
def calcNewCenterMpo(O, Ac, C, Fl, Fr, lam, tol=1e-5):
    """
    Find new guess for Ac and C as fixed points of the maps O_Ac and O_C.
    
        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Fl : np.array (D, d, D)
            left environment,
            ordered bottom-middle-top.
        Fr : np.array (D, d, D)
            right environment,
            ordered top-middle-bottom.
        lam : float
            Leading eigenvalue.
        tol : float, optional
            current tolerance
    
        Returns
        -------
        AcTilde : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        CTilde : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.

    """
    
    return AcTilde, CTilde

Since the `minAcC` routine to extract a new set of mixed gauge MPS tensors from the updated $\tilde{A}_C$ and $\tilde{C}$ can be reused from the previous chapter, we now have all the tools needed to implement the VUMPS algorithm for MPOs.

In [ ]:
def vumpsMpo(O, D, A0=None, tol=1e-4, tolFactor=1e-2, verbose=True):
    """
    Find the fixed point MPS of a given MPO using VUMPS.
    
        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        D : int
            Bond dimension
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Relative convergence criterium.
        
        Returns
        -------
        lam : float
            Leading eigenvalue.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Fl : np.array (D, d, D)
            left environment,
            ordered bottom-middle-top.
        Fr : np.array (D, d, D)
            right environment,
            ordered top-middle-bottom.
        
    """
    
    return lam, Al, Ac, Ar, C, Fl, Fr

### 3.2 The two-dimensional classical Ising model

Next we apply the VUMPS algorithm for MPOs to the two-dimensional classical Ising model. To this end, consider classical spins $s_i = \pm 1$ placed on the sites of an infinite square lattice which interact according to the nearest-neighbor Hamiltonian

$$H = -J \sum_{\langle i,j \rangle} s_i s_j \,.$$

We now wish to compute the corresponding partition function,

$$ \mathcal{Z} = \sum_{\{s_i\}} \text{e}^{-\beta H({\{s_i\}})},$$

using our freshly implemented algorithm. In order to do this we first rewrite this partition function as the contraction of an infinite two-dimensional tensor network,

<center><img src="./img/Z.svg" alt="Z"/></center>

where every edge in the network has bond dimension $d = 2$. Here, the black dots represent $ \delta $-tensors

<center><img src="./img/delta.svg" alt="delta"/></center>

and the matrices $t$ encode the Boltzmann weights associated to each nearest-neighbor interaction,

<center><img src="./img/t.svg" alt="t"/></center>

In order to arrive at a translation invariant network corresponding to a single hermitian MPO tensor we can take the matrix square root $q$ of each Boltzmann matrix such that

<center><img src="./img/q.svg" alt="q"/></center>

and absorb the result symmetrically into the $\delta$-tensors at each vertex to define the MPO tensor

<center><img src="./img/O.svg" alt="O"/></center>

The partition function then becomes

<center><img src="./img/Z2.svg" alt="Z2p"/></center>

which precisely corresponds to an infinite power of a row-to-row transfer matrix $T(O)$ of the kind defined above. We can therefore use the VUMPS algorithm to determine its fixed point, where the corresponding eigenvalue automatically gives us the free energy density as explained before.

Having found this fixed point and its corresponding environments, we can easily evaluate expectation values of local observables. For example, say we want to find the expectation value of the magnetization at site $\mu$,

$$ \langle m \rangle = \frac{1}{\mathcal{Z}} \sum_{\{s_i\}} s_\mu \text{e}^{-\beta H({\{s_i\}})}.$$

We can access this quantity by introducing a magnetization tensor $M$, placing it at site $\mu$ and contracting the partition function network around it as

<center><img src="./img/Mexp.svg" alt="Mexp"/></center>

where the normalization factor $\mathcal{Z}$ in the denominator is taken care of by the same contraction where $O$ is left at site $\mu$ (which in this case is of course nothing more than the eigenvalue $\lambda$). The magnetization tensor $M$ is defined entirely analogously to the MPO tensor $O$, but where instead of a regular $\delta$-tensor the entry $i=j=k=l=1$ (using base-0 indexing) is set to $-1$ instead of $1$.

We can now define the routines for constructing the Ising MPO and magnetization tensor an computing local expectation values, as well as a routine that implements [Onsager's exact solution for this model](https://en.wikipedia.org/wiki/Ising_model#Onsager's_exact_solution) to compare our results to.

In [ ]:
def isingO(beta, J):
    """
    Gives the MPO tensor corresponding to the partition function of the 2d 
    classical Ising model at a given temperature and coupling, obtained by
    distributing the Boltzmann weights evenly over all vertices.
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        O : np.array (2, 2, 2, 2)
            MPO tensor,
            ordered left-top-right-bottom.

    """
    # basic vertex delta tensor
    vertex = np.zeros( (2,) * 4 )
    vertex[tuple([np.arange(2)] * 4)] = 1
    # take matrix square root of Boltzmann weights and pull into vertex edges
    t = np.array([[np.exp(beta*J), np.exp(-beta*J)], [np.exp(-beta*J), np.exp(beta*J)]])
    q = sqrtm(t)
    O = ncon((q, q, q, q, vertex), ([-1, 1], [-2, 2], [-3, 3], [-4, 4], [1, 2, 3, 4]))
    return O


def isingM(beta, J):
    """
    Gives the magnetizatopn MPO tensor for the 2d classical Ising model at a
    given temperature and coupling.
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        M : np.array (2, 2, 2, 2)
            Magnetization tensor,
            ordered left-top-right-bottom.

    """
    # basic vertex delta tensor
    vertex = np.zeros( (2,) * 4 )
    vertex[tuple([np.arange(2)] * 4)] = 1
    # change sign of (1, 1, 1, 1) entry
    vertex[1, 1, 1, 1] = -1;
    # take matrix square root of Boltzmann weights and pull into vertex edges
    t = np.array([[np.exp(beta*J), np.exp(-beta*J)], [np.exp(-beta*J), np.exp(beta*J)]])
    q = sqrtm(t)
    M = ncon((q, q, q, q, vertex), ([-1, 1], [-2, 2], [-3, 3], [-4, 4], [1, 2, 3, 4]))
    return M


def expValMpo(O, Ac, Fl, Fr):
    """
    Gives the MPO tensor corresponding to the partition function of the 2d 
    classical Ising model at a given temperature and coupling, obtained by
    distributing the Boltzmann weights evenly over all vertices.
    
        Parameters
        ----------
        O : np.array (2, 2, 2, 2)
            local operator of which we want to
            compute the expectation value,
            ordered left-top-right-bottom.
        Ac : np.array (D, d, D)
            MPS tensor of the MPS fixed point,
            with 3 legs ordered left-bottom-right,
            center gauge.
        Fl : np.array (D, d, D)
            left environmnt,
            ordered bottom-middle-top.
        Fr : np.array (D, d, D)
            right environmnt,
            ordered top-middle-bottom.
    
        Returns
        -------
        e : float
            expectation value of the operator O.
        

    """
    e = ncon((Fl, Ac, O, np.conj(Ac), Fr), (
            [1, 3, 2], [2, 7, 5], [3, 7, 8, 6], [1, 6, 4], [5, 8, 4]))
    
    return e


def isingExact(beta, J):
    """
    Exact Onsager solution for the 2d classical Ising Model

        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        magnetization : float
            Magnetization at given temperature and coupling.
        free : float
            Free energy at given temperature and coupling.
        energy : float
            Energy at given temperature and coupling.

    """
    theta = np.arange(0, np.pi/2, 1e-6)
    x = 2 * np.sinh(2 * J * beta) / np.cosh(2 * J * beta) ** 2
    if 1 - (np.sinh(2 * J * beta)) ** (-4) > 0:
        magnetization = (1 - (np.sinh(2 * J * beta)) ** (-4)) ** (1 / 8)
    else:
        magnetization = 0
    free = -1 / beta * (np.log(2 * np.cosh(2 * J * beta)) + 1 / np.pi * np.trapz(np.log(1 / 2 * (1 + np.sqrt(1 - x ** 2 * np.sin(theta) ** 2))), theta))
    K = np.trapz(1 / np.sqrt(1 - x ** 2 * np.sin(theta) ** 2), theta)
    energy = -J * np.cosh(2 * J * beta) / np.sinh(2 * J * beta) * (1 + 2 / np.pi * (2 * np.tanh(2 * J * beta) ** 2 - 1) * K)
    return magnetization, free, energy

We can now demonstrate the VUMPS algorithm for MPOs. We will fix $J = 1$ in the following, and investigate the behavior of the model as a function of temperature. Since we know the critical piont is located at $T_c = \frac{2}{\log\left(1 + \sqrt{2}\right)} \approx 2.26919$, let us first have a look at $T = 4$ and $T = 1$ far above and below the critical temperature, for which we expect a vanishing and non-vanishing magnetization respectively.

In [ ]:
D = 12
d = 2
J = 1
tol = 1e-8
tolFactor = 1e-4
A0 = createMPS(D, d)

T = 4
print('Running for T = {}\n'.format(T))
beta = 1 / T
O = isingO(beta, J)
M = isingM(beta, J)
lam, Al, Ac, Ar, C, Fl, Fr = vumpsMpo(O, D, A0=A0, tol=tol, tolFactor=tolFactor, verbose=True)
mag = np.abs(expValMpo(M, Ac, Fl, Fr)/expValMpo(O, Ac, Fl, Fr))
magExact = isingExact(beta, J)[0]
freeEnergy = -np.log(lam)/beta
freeEnergyExact = isingExact(beta, J)[1]
print('\nFree energy: {:.10f}; \trelative difference with exact solution: {:.4e}\n'.format(
                freeEnergy, np.abs((freeEnergy - freeEnergyExact) / freeEnergyExact)))
print('Magnetization: {:.10f}\n'.format(mag))

T = 1
print('Running for T = {}\n'.format(T))
beta = 1 / T
O = isingO(beta, J)
M = isingM(beta, J)
lam, Al, Ac, Ar, C, Fl, Fr = vumpsMpo(O, D, A0=A0, tol=tol, tolFactor=tolFactor, verbose=True)
mag = np.abs(expValMpo(M, Ac, Fl, Fr)/expValMpo(O, Ac, Fl, Fr))
magExact = isingExact(beta, J)[0]
freeEnergy = -np.log(lam)/beta
freeEnergyExact = isingExact(beta, J)[1]
print('\nFree energy: {:.10f}; \trelative difference with exact solution: {:.4e}\n'.format(
                freeEnergy, np.abs((freeEnergy - freeEnergyExact) / freeEnergyExact)))
print('Magnetization: {:.10f}; \trelative difference with exact solution: {:.4e}\n'.format(
                mag, np.abs((mag - magExact) / magExact)))


We clearly see that far from the critical point the VUMPS algorithm achieves excellent agreement with the exact solution efficiently at very small bond dimensions.

As a final demonstration, we compute the magnetization and free energy over a range from $T = 1$ to $T = 3.4$ and plot the results. Note that convergence of the algorithm slows down significantly near the critical point, as can be expected.

In [ ]:
D = 12
d = 2
J = 1

print('Bond dimension: D = {}\n'.format(D))
Al = createMPS(D, d)
# optimization parameters
tol = 1e-8
tolFactor = 1e-2
verbose = False

Ts = np.linspace(1., 3.4, 100)
magnetizations = []
magnetizationsExact = []
freeEnergies = []
freeEnergiesExact = []

for T in Ts:
    beta = 1/T
    O = isingO(beta, J)
    M = isingM(beta, J)
    print('Running for T = {:.5f}'.format(T), end="\r")
    
    lam, Al, Ac, Ar, C, Fl, Fr = vumpsMpo(O, D, A0=Al, tol=tol, tolFactor=tolFactor, verbose=verbose)
    magnetizations.append(np.abs(expValMpo(M, Ac, Fl, Fr)/expValMpo(O, Ac, Fl, Fr)))
    magnetizationsExact.append(isingExact(beta, J)[0])
    freeEnergies.append(-np.log(lam)/beta)
    freeEnergiesExact.append(isingExact(beta, J)[1])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5), dpi=120)
ax1.set_title('Magnetization as a function of the temperature')
ax1.set(xlabel=r'$T$', ylabel=r'$<M>$')
ax1.scatter(Ts, magnetizations, label = 'D={}'.format(D), c="hotpink", marker="^")
ax1.plot(Ts, magnetizationsExact, label = 'exact')

ax2.set_title('Free energy as a function of the temperature')
ax2.set(xlabel=r'$T$', ylabel=r'$f$')
ax2.scatter(Ts, freeEnergies, label = 'D={}'.format(D), c="hotpink", marker="^")
ax2.plot(Ts, freeEnergiesExact, label = 'exact')
plt.legend()
